In [1]:
# we either import locally (after the repo was cloned)
# or we first clone it and then use it (e.g., when run in Google Colab)
shell = get_ipython().__class__.__name__ 

if shell == 'Shell':
    # imports when launched in e.g., Google Colab
    !git clone https://github.com/filipzz/athena.git r2b2
    from r2b2.code.athena.athena import AthenaAudit
    from r2b2.code.athena.contest import Contest
    from r2b2.code.athena.audit import Audit
else: # shell ==  'ZMQInteractiveShell' or shell == 'TerminalInteractiveShell'
    # local imports if you run it with e.g., Jupyter
    from athena.athena import AthenaAudit
    from athena.contest import Contest
    from athena.audit import Audit

import pandas as pd

In [2]:
# risk limit of the audit
risk_limit = 0.1 

In [3]:
# address of the file with election results
#election_results = "https://raw.githubusercontent.com/gwexploratoryaudits/brla_explore/poorvi/R2Audits/Athena/Scripts/2020MontgomeryPrimary/2020_montgomery_formatted.json"
election_results = "tmp/2020_montgomery_formatted.json"

w = Audit("athena", risk_limit)

w.read_election_results(election_results)

#w.get_contests()

In [4]:
contest = "d_president"

w.load_contest(contest)

In [5]:
#w.election.print_election()
w.show_election_results()

,Candidates,Results
0,Bennet,49
1,Biden,28135
2,Bloomberg,682
3,Buttigieg,509
4,Gabbard,130
5,Klobuchar,396
6,Patrick,23
7,Sanders,5466
8,Steyer,57
9,Warren,1074


In [6]:
w.run_audit_round()
w.present_state()



---------------------- Round number:  1  -----------------

Your choices: 
[1] Find next round size at 70%, 80%, 90%
[2] Enter other goal for probability of stopping.
Enter your choice: 1

Select round size: 
Complete with prob. 0.7 when you sample 21 more ballots.
Complete with prob. 0.8 when you sample 25 more ballots.
Complete with prob. 0.9 when you sample 29 more ballots.


	Enter the number of (all) ballots drawn in this round: 40
	Enter the number of relevant ballots: 40

	Enter number of ballots for each candidate:
	Ballots for Bennet: 1
	Ballots for Biden: 15
	Ballots for Bloomberg: 1
	Ballots for Buttigieg: 4
	Ballots for Gabbard: 1
	Ballots for Klobuchar: 1
	Ballots for Patrick: 1
	Ballots for Sanders: 8
	Ballots for Steyer: 1
	Ballots for Warren: 6
	Ballots for Write_Ins: 1
[0.009008703217523537]
[0.0009234474078164121]
[25.031956574310442]
[0.003555140439678134]
[0.0013557917641022668]
[0.018911122791365746]
[3.4861811393987145]
[0.007779570289495356]
[338.44731532415614

,Candidates,Results,Round 1,Total,Required
0,Bennet,49,1,1,
1,Biden,28135,15,15,17
2,Bloomberg,682,1,1,
3,Buttigieg,509,4,4,
4,Gabbard,130,1,1,
5,Klobuchar,396,1,1,
6,Patrick,23,1,1,
7,Sanders,5466,8,8,
8,Steyer,57,1,1,
9,Warren,1074,6,6,


In [ ]:
w.run_audit_round()
w.present_state()

### DEV part

In [ ]:
d = {'Candidates': w.election.candidates, 'Results': w.election.results}
df = pd.DataFrame(data=d)

# Results of round samples
for rd in range(w.round_number):
    rd_caption = "Round " + str(rd + 1)
    r = []
    for i in range(len(w.election.candidates)):
        r.append(w.round_observations[i][rd])
    df[rd_caption] = r

# Total column - sum of sampled ballots    
r = []
rd_caption = "Total"
for i in range(len(w.election.candidates)):
    r.append(w.audit_observations[i][w.round_number - 1])
df[rd_caption] = r

# Required column - presents kmin values to pass the audit
r = []
rd_caption = "Required"
for i in range(len(w.election.candidates)):
    if w.min_kmins[i] == 0:
        r.append("")
    else:
        r.append(w.min_kmins[i])
df[rd_caption] = r

s = df.style.set_properties(subset = pd.IndexSlice[w.election.winners, :], **{'color' : 'green'})


# This part is only needed if you are not planning to enter election details "by hand"


In [ ]:
audit_type = "athena" 

In [ ]:
logging.getLogger(__name__).setLevel("INFO")

In [ ]:
alpha = 0.1 # risk limit
name = "Election name"
winners = 1 # number of winners

In [ ]:
results = [17800, 9700] # ballots for candidates
total = 67000 # total number of ballots cast (can be set to 0)

In [ ]:
delta = 1
candidates = [string.ascii_uppercase[i] for i in range(len(results))]
ballots_cast = total
if ballots_cast < sum(results):
    ballots_cast = sum(results)
model = "bin"
round_schedule = []
election = {}
election["ballots_cast"] = ballots_cast
election["alpha"] = alpha
election["delta"] = delta
election["candidates"] = candidates
election["results"] = results
election["winners"] = winners
election["name"] = name
election["model"] = model
election_object = Election(election)
election_object.print_election()

In [ ]:
w = Audit(audit_type, alpha, delta)
w.add_election(election)
w.add_round_schedule(round_schedule)

w.run_interactive()
